Aldao Amoedo, Héctor

Cabaleiro Pintos, Laura

Cotardo Valcárcel, Donato José

Romero Conde, José

---

##   _Librerias_

In [2]:
import Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Statistics")
Pkg.add("StatsBase")
Pkg.add("Random")
Pkg.add("ScikitLearn")
Pkg.add("Plots")
Pkg.add("MLBase")

   Resolving package versions...
  No Changes to `C:\Users\Usuario\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Usuario\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Usuario\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Usuario\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Usuario\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Usuario\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Usuario\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Usuario\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Usuario\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Usuario\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Usuario\.julia\environments\v1.

In [45]:
using CSV
using DataFrames
using Plots
using Statistics
using Random
using ScikitLearn
using ScikitLearn.Pipelines: Pipeline, named_steps, FeatureUnion
using ScikitLearn.GridSearch: GridSearchCV 
using StatsBase: mode
@sk_import decomposition: (PCA, FastICA)
@sk_import discriminant_analysis: LinearDiscriminantAnalysis
@sk_import ensemble: (AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier)
@sk_import feature_selection: (SelectKBest, f_classif, mutual_info_classif, RFE)
@sk_import impute: SimpleImputer
@sk_import linear_model: LogisticRegression
@sk_import manifold: (LocallyLinearEmbedding, Isomap)
@sk_import neighbors: KNeighborsClassifier
@sk_import neural_network: MLPClassifier
@sk_import preprocessing: MinMaxScaler
@sk_import svm: SVC

PyObject <class 'sklearn.svm._classes.SVC'>

## Preparación de los datos

El punto 1, está completado, aunque creo que el 1.3, donde preparamos los datos transformandolos y rellanando los nulos está mal.

### 1. Cargar los datos y descripción

In [2]:
df = CSV.read("Datos_Practica_Evaluacion_1.csv", DataFrame)

num_instancias, num_variables = size(df)
num_individuos = length(unique(df[:, 1]))
num_clases_salida = length(unique(df[:, end]))

println("Número de variables: $num_variables")
println("Número de instancias: $num_instancias")
println("Número de individuos: $num_individuos")
println("Número de clases de salida: $num_clases_salida")

Número de variables: 563
Número de instancias: 10299
Número de individuos: 30
Número de clases de salida: 6


Los datos dados ya están cargados, y como podemos observar tienen:
* 563 Variables
* 10299 Instancias
* 30 Individuos
* 6 Clases de salida

### 2. Calcular porcentaje de nulos

In [3]:
num_nulos_totales = 0

for col ∈ names(df)
    num_nulos = count(ismissing, df[:, col])
    num_nulos_totales += num_nulos
    porcentaje_nulos = (num_nulos / num_instancias) * 100
end

porcentaje_nulos_totales = (num_nulos_totales / (num_instancias * num_variables)) * 100
println("Porcentaje total de nulos en el conjunto: $porcentaje_nulos_totales%")  

Porcentaje total de nulos en el conjunto: 0.004656507546905259%


### 3. Preparar los datos para las técnicas de clasificación

Para rellenar valores faltantes tenemos que hacernos una idea de que tipo de datos encontraremos.

In [4]:
for col ∈ names(df)
    if eltype(df[!, col]) == Union{Missing, Float64}
        df[ismissing.(df[!, col]), col] .= mean(skipmissing(df[!, col]))
        df[!, col] = Float64.(df[!, col]) 
    end
end

println("Valores nulos rellenados.")

Valores nulos rellenados.


In [5]:
Set([eltype(df[!,col]) for col in names(df)])  # ya no hay missing

Set{DataType} with 3 elements:
  Float64
  String31
  Int64

### 4. Segmentar el 10% de los datos usando HoldOut

In [6]:
Random.seed!(172)

holdout_individuos = shuffle(unique(df[:, :subject]))[1:Int(round(0.1 * length(unique(df[:, :subject]))))]  
holdout_df = filter(fila -> fila.subject in holdout_individuos, df)
train_df = filter(fila -> !(fila.subject in holdout_individuos), df)

println("Individuos en el holdout: ", holdout_individuos)
println("Tamaño del conjunto de entrenamiento: $(size(train_df)[1])")
println("Tamaño del conjunto de holdout: $(size(holdout_df)[1])")

Individuos en el holdout: [1, 16, 9]
Tamaño del conjunto de entrenamiento: 9298
Tamaño del conjunto de holdout: 1001


### 5. Fold y escalado

In [7]:
# separacion X Y
train = Array(train_df)
test = Array(holdout_df)

X_train = train[:,1:end-1]
Y_train = train[:,end]
X_test = test[:,1:end-1]
Y_test = test[:,end];
# k fold
especificacionCV = ScikitLearn.CrossValidation.KFold(size(train)[1], n_folds=5)
#folds = [(train[indicesTrain,:], train[indicesTest,:]) for (indicesTrain, indicesTest) ∈ especificacionCV]
# escalado
X_train = fit_transform!(MinMaxScaler(), X_train);
X_test = fit_transform!(MinMaxScaler(), X_test);
#folds = [((fit_transform!(MinMaxScaler(), train[:,1:end-1]),Vector(train[:,end])), (fit_transform!(MinMaxScaler(), test[:,1:end-1]),Vector(test[:,end]))) for (train, test) ∈ folds ];

# de esta forma:
# folds tiene los 5 folds (escalado cada uno independientemente)
# folds[1] es un fold
# folds[1][1] es el entrenamiento de ese fold
# folds[1][1][1] es el X del entrenamiento de ese fold

1001×562 Matrix{Float64}:
 0.0  0.672712  0.760056  0.58034   …  0.0767163  0.789732  0.703263
 0.0  0.659042  0.76925   0.59601      0.135843   0.648623  0.635692
 0.0  0.661902  0.766977  0.594195     0.135882   0.649687  0.635003
 0.0  0.662402  0.765118  0.595333     0.136108   0.650154  0.634467
 0.0  0.660272  0.767131  0.596341     0.135625   0.651012  0.634509
 0.0  0.662521  0.763664  0.595765  …  0.135792   0.651288  0.634159
 0.0  0.660466  0.771552  0.595979     0.135112   0.653333  0.633753
 0.0  0.660854  0.768034  0.598375     0.134084   0.654766  0.634105
 0.0  0.661492  0.764832  0.595514     0.134031   0.655338  0.633848
 0.0  0.657451  0.776695  0.596657     0.132291   0.658889  0.633882
 ⋮                                  ⋱             ⋮         
 1.0  0.678907  0.77363   0.595371     0.260264   0.876956  0.978448
 1.0  0.629944  0.74135   0.575188     0.262351   0.877013  0.980838
 1.0  0.6685    0.767715  0.565007     0.0523835  0.683994  0.81228
 1.0  0.656791  0

---

# Modelos Basicos

In [8]:
function Mejores_parametros(df)
    clasificadores = unique(df[!,:clasificador])
    println("MEJORES RESULTADOS POR CLASIFICADOR:")
    for nombreClasificador in clasificadores
        df_clasificador = filter(fila -> fila.clasificador == nombreClasificador, df)
        indice = argmax(df_clasificador[!, Symbol("Accuracy")])
        mejor_fila = df_clasificador[indice, :]
        println("Clasificador: ", mejor_fila.clasificador)
        println("Precisión: ", mejor_fila.Accuracy)
        println("Parámetros: ", mejor_fila.parametros)
    end
end


Mejores_parametros (generic function with 1 method)

---

In [9]:
function plot_transformed_data(name::String, reductor, X, y)

    X_reducida = fit_transform!(reducer, X, y)
    plot = scatter(X_reduced[:, 1], X_reduced[:, 2], group=y, legend=:topright, title=name,
                xlabel="Componente 1", ylabel="Componente 2", markersize=5)
    return plot
end

plot_transformed_data (generic function with 1 method)

In [11]:
resultadosModelosBasicos = DataFrame(filtrado = String[], reduccion = String[], clasificador = String[], Accuracy = Float64[], parametros = String[])
plot_distribucion = @layout [a b c; d e]

filtrado = Dict(
   #"nada" => "passthrough",
   "anova" => SelectKBest(score_func=f_classif),
   "mi" => SelectKBest(score_func=mutual_info_classif),
   "rfe" => RFE(LogisticRegression(max_iter=10),step=0.5)
 )

reduccion = Dict(
   #"nada" => "passthrough",
   "pca" => PCA(),
   "lda" => LinearDiscriminantAnalysis(),
   "ica" => FastICA(),
   #"isomap" => Isomap(n_neighbors=25),
   #"lle" => LocallyLinearEmbedding(),
 )

clasificacion = Dict(
    "mlp" => [MLPClassifier(max_iter=10), Dict(:classifier__hidden_layer_sizes => [[50], [100], [100, 50]])],
    "knn" => [KNeighborsClassifier(), Dict(:classifier__n_neighbors =>[1, 10, 20])],
    "svm" => [SVC(), Dict(:classifier__C =>[0.1, 1, 10])]
)

for (nombreFiltro, filtro) in filtrado
    for (nombreReduccion, reduccion) in reduccion
        for (nombreClasificador, valor) in clasificacion

            clasificador = valor[1]
            parametros = valor[2]
            modelo = Pipeline([
                ("filtro", filtro),
                ("reduccion", reduccion),
                ("classifier", clasificador) 
            ])

            busqueda = GridSearchCV(modelo, parametros, cv=especificacionCV)
            fit!(busqueda, X_train, Y_train)
            mejorModelo = busqueda.best_estimator_
            mejoresParametros = busqueda.best_params_
            accuracy = busqueda.best_score_
            Y_pred = predict(mejorModelo, X_test)
            accuracy = sum(Y_pred .== Y_test) / length(Y_test)
            push!(resultadosModelosBasicos, (nombreFiltro, nombreReduccion, nombreClasificador, accuracy, string(mejoresParametros)))
            println("Filtrado: $nombreFiltro,\nReducción: $nombreReduccion,\nClasificador: $nombreClasificador,\nparámetros: $(mejoresParametros),\nPrecisión: $accuracy\n")
        end
    end
end

Mejores_parametros(resultadosModelosBasicos)


Filtrado: anova,
Reducción: lda,
Clasificador: knn,
parámetros: Dict{Symbol, Any}(:classifier__n_neighbors => 20),
Precisión: 0.7512487512487512

Filtrado: anova,
Reducción: lda,
Clasificador: mlp,
parámetros: Dict{Symbol, Any}(:classifier__hidden_layer_sizes => [100]),
Precisión: 0.7282717282717283



C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\.ju

Filtrado: anova,
Reducción: lda,
Clasificador: svm,
parámetros: Dict{Symbol, Any}(:classifier__C => 1.0),
Precisión: 0.7602397602397603

Filtrado: anova,
Reducción: ica,
Clasificador: knn,
parámetros: Dict{Symbol, Any}(:classifier__n_neighbors => 20),
Precisión: 0.7092907092907093



C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Filtrado: anova,
Reducción: ica,
Clasificador: mlp,
parámetros: Dict{Symbol, Any}(:classifier__hidden_layer_sizes => [100, 50]),
Precisión: 0.7972027972027972



...


Filtrado: mi,
Reducción: ica,
Clasificador: svm,
parámetros: Dict{Symbol, Any}(:classifier__C => 10.0),
Precisión: 0.5014985014985015


C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(



Filtrado: mi,
Reducción: pca,
Clasificador: knn,
parámetros: Dict{Symbol, Any}(:classifier__n_neighbors => 20),
Precisión: 0.6163836163836164

Filtrado: mi,
Reducción: pca,
Clasificador: mlp,
parámetros: Dict{Symbol, Any}(:classifier__hidden_layer_sizes => [100, 50]),
Precisión: 0.6233766233766234


C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\.ju


Filtrado: mi,
Reducción: pca,
Clasificador: svm,
parámetros: Dict{Symbol, Any}(:classifier__C => 10.0),
Precisión: 0.5234765234765235

MEJORES RESULTADOS POR CLASIFICADOR:
Clasificador: knn
Precisión: 0.8291708291708292
Parámetros: Dict{Symbol, Any}(:classifier__n_neighbors => 10)
Clasificador: mlp
Precisión: 0.8981018981018981
Parámetros: Dict{Symbol, Any}(:classifier__hidden_layer_sizes => [100, 50])
Clasificador: svm
Precisión: 0.8901098901098901
Parámetros: Dict{Symbol, Any}(:classifier__C => 10.0)


**MEJORES RESULTADOS POR CLASIFICADOR:**

**Clasificador: knn**
- Precisión: 0.8291708291708292
- Parámetros: Dict{Symbol, Any}(:classifier__n_neighbors => 10)

**Clasificador: mlp**
- Precisión: 0.8981018981018981
- Parámetros: Dict{Symbol, Any}(:classifier__hidden_layer_sizes => [100, 50])

**Clasificador: svm**
- Precisión: 0.8901098901098901
- Parámetros: Dict{Symbol, Any}(:classifier__C => 10.0)

---

# Ensembles

---

10. Adicionalmente, con los datos sólo con el tratamiento de Filtrado ANOVA, recrear
las siguientes técnicas
 - BaggingClassifier con clasificador base KNN con número de vecinos 5 y
número de estimadores 10 y 50
 -  AdaBoosting con estimadores SVM con kernel lineal siendo el número de
estimadores 5.
 -  GBM (GradientBoostingClasifier), con 50 estimadores y un learning_rate de
0.2

In [18]:
resultadosEjercicio10 = DataFrame(clasificador = String[], Accuracy = Float64[], parametros = String[])

clasificadoresEjercicio10 = Dict(
    "bagging10" => BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=5), n_estimators=10),
    "bagging50" => BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=5), n_estimators=50),
    "adaboosting" => AdaBoostClassifier(estimator=SVC(kernel="linear"), algorithm="SAMME", n_estimators=5),
    "gbm" => GradientBoostingClassifier(learning_rate=0.2, n_estimators=50)
)

for (nombreClasificador, clasificador) in clasificadoresEjercicio10
    modelo = Pipeline([
        ("filtro", SelectKBest(score_func=f_classif)),
        ("classifier", clasificador)
    ])

    fit!(modelo, X_train, Y_train)
    Y_pred = predict(modelo, X_test)
    accuracy = sum(Y_pred .== Y_test) / length(Y_test)
    push!(resultadosEjercicio10, (nombreClasificador, accuracy, string(clasificador.get_params())))
    println("Clasificador: $nombreClasificador Precisión: $accuracy")
end

Mejores_parametros(resultadosEjercicio10)


Clasificador: bagging50 Precisión: 0.7202797202797203
Clasificador: bagging10 Precisión: 0.7062937062937062
Clasificador: adaboosting Precisión: 0.16983016983016982
Clasificador: gbm Precisión: 0.7632367632367633
MEJORES RESULTADOS POR CLASIFICADOR:
Clasificador: bagging50
Precisión: 0.7202797202797203
Parámetros: Dict{Any, Any}("estimator__weights" => "uniform", "max_features" => 1.0, "estimator__metric_params" => nothing, "estimator" => PyObject KNeighborsClassifier(), "verbose" => 0, "estimator__leaf_size" => 30, "estimator__n_jobs" => nothing, "estimator__n_neighbors" => 5, "oob_score" => false, "max_samples" => 1.0, "estimator__algorithm" => "auto", "estimator__p" => 2, "n_jobs" => nothing, "warm_start" => false, "random_state" => nothing, "bootstrap" => true, "n_estimators" => 50, "bootstrap_features" => false, "estimator__metric" => "minkowski")
Clasificador: bagging10
Precisión: 0.7062937062937062
Parámetros: Dict{Any, Any}("estimator__weights" => "uniform", "max_features" => 1

In [22]:
Mejores_parametros(resultadosEjercicio10)

MEJORES RESULTADOS POR CLASIFICADOR:
Any["bagging50", 0.7102897102897103]
Any["bagging10", 0.7112887112887113]
Any["adaboosting", 0.16983016983016982]
Any["gbm", 0.7702297702297702]


---

11. Entrenar con el conjunto completo de entrenamiento (todo lo que componía el 5-
fold cross-validation) y testear son el 10% reservado
 - Coger las 5 mejores combinaciones de los modelos anteriores de
clasificación, (1 KNN, 1 SVM, 1 MLP, 1 Bagging y 1 AdaBoosting)
 - Crear un Random Forest con valor para los estimadores del 500 y
profundidad máxima de 10
 - Crear un Hard Voting con las mejores combinaciones del KNN, SVM y MLP
(uno para cada una de las técnicas)
 - Crear un Soft Voting con las mejores combinaciones del KNN, SVM y MLP
(uno para cada una de las técnicas) para los pesos coger el porcentaje de
acierto en test de cada una de las combinaciones en el 5-fold cross-
valiadation
 - Crear un Ensemble Stacking con MLP como clasificador final, así mismo,
use como base las mejores combinaciones del SVM, KNN y MLP
 - Crear un XGBoost con los valores por defecto
 - Crear un LightGBM, con los valores por defecto
 - Crear un Catboost, con los valores por defecto

In [22]:
# Ejercicio 11

using DecisionTree, ScikitLearn, XGBoost, LightGBM, CatBoost
using DataFrames, Statistics
using ScikitLearn.CrossValidation: KFold
using ScikitLearn: fit!, predict 


# Los mejores modelos obtenidos previamente
mejor_knn = KNeighborsClassifier(n_neighbors=10) 
mejor_svm = SVC(C=10.0)
mejor_mlp = MLPClassifier(hidden_layer_sizes=[100, 50])
mejor_bagging = BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=5), n_estimators=50)
mejor_adaboost = AdaBoostClassifier(estimator=SVC(kernel="linear", probability=true), n_estimators=5)


# Entrenar los mejores modelos en todo el conjunto de entrenamiento
modelos = Dict(
    "KNN" => mejor_knn,
    "SVM" => mejor_svm,
    "MLP" => mejor_mlp,
    "Bagging" => mejor_bagging,
    "AdaBoost" => mejor_adaboost
)

for (nombre, modelo) in modelos
    fit!(modelo, X_train, Y_train)
    predicciones = predict(modelo, X_test)
    accuracy = sum(predicciones .== Y_test) / length(Y_test)
    println("Modelo: $nombre, Accuracy en test: $accuracy")
end



Modelo: SVM, Accuracy en test: 0.906093906093906
Modelo: Bagging, Accuracy en test: 0.8411588411588412
Modelo: MLP, Accuracy en test: 0.8861138861138861
Modelo: AdaBoost, Accuracy en test: 0.8191808191808192

C:\Users\Usuario\.julia\conda\3\x86_64\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Modelo: KNN, Accuracy en test: 0.8501498501498501


In [26]:
# Crear un Random Forest
rf = RandomForestClassifier(n_trees=500, max_depth=10)
fit!(rf, X_train, Y_train)
rf_pred = predict(rf, X_test)
rf_accuracy = sum(rf_pred .== Y_test) / length(Y_test)
println("Random Forest Accuracy: $rf_accuracy")

Random Forest Accuracy: 0.7972027972027972


In [61]:
# Hard voting

function hard_voting(models, X_test)
    predicciones = [predict(model, X_test) for model in values(models)]
    combined = [mode([pred[i] for pred in predicciones]) for i in 1:size(X_test, 1)]
    return combined
end

hard_voting_models = Dict(
    "KNN" => mejor_knn,
    "SVM" => mejor_svm,
    "MLP" => mejor_mlp,
    "Bagging" => mejor_bagging,
    "AdaBoost" => mejor_adaboost
) 

hard_voting_pred = hard_voting(hard_voting_models, X_test)
hard_voting_accuracy = sum(hard_voting_pred .== Y_test) / length(Y_test)
println("Hard Voting Accuracy: $hard_voting_accuracy")

#Soft voting

#BRU como cojones se importan cosas aiuda fk fk fk

Hard Voting Accuracy: 0.8871128871128872


In [55]:
#Mismo fking problema aaaaaaaaaaaaaaaaaa

# XGBoost
xgboost = xgboost.XGBClassifier()
fit!(xgboost, X_train, Y_train)
xgboost_pred = predict(xgboost, X_test)
xgboost_accuracy = sum(xgboost_pred .== Y_test) / length(Y_test)
println("XGBoost Accuracy: $xgboost_accuracy")

# LightGBM
lightgbm = lightgbm.LGBMClassifier()
fit!(lightgbm, X_train, Y_train)
lightgbm_pred = predict(lightgbm, X_test)
lightgbm_accuracy = sum(lightgbm_pred .== Y_test) / length(Y_test)
println("LightGBM Accuracy: $lightgbm_accuracy")

# CatBoost
catboost = catboost.CatBoostClassifier()
fit!(catboost, X_train, Y_train)
catboost_pred = predict(catboost, X_test)
catboost_accuracy = sum(catboost_pred .== Y_test) / length(Y_test)
println("CatBoost Accuracy: $catboost_accuracy")

   Resolving package versions...
    Updating `C:\Users\Usuario\.julia\environments\v1.10\Project.toml`
  [438e738f] + PyCall v1.96.4
  No Changes to `C:\Users\Usuario\.julia\environments\v1.10\Manifest.toml`


PyCall.PyError: PyError (PyImport_ImportModule

The Python package xgboost could not be imported by pyimport. Usually this means
that you did not install xgboost in the Python version being used by PyCall.

PyCall is currently configured to use the Julia-specific Python distribution
installed by the Conda.jl package.  To install the xgboost module, you can
use `pyimport_conda("xgboost", PKG)`, where PKG is the Anaconda
package that contains the module xgboost, or alternatively you can use the
Conda package directly (via `using Conda` followed by `Conda.add` etcetera).

Alternatively, if you want to use a different Python distribution on your
system, such as a system-wide Python (as opposed to the Julia-specific Python),
you can re-configure PyCall with that Python.   As explained in the PyCall
documentation, set ENV["PYTHON"] to the path/name of the python executable
you want to use, run Pkg.build("PyCall"), and re-launch Julia.

) <class 'ModuleNotFoundError'>
ModuleNotFoundError("No 

## Conclusiones

12. Imprima la importancia de las variables seleccionadas por el Random Forest y por
el XGBoost ordenadas por importancia.

In [47]:
# Importancia de variables para Random Forest
importancias_rf = rf
indices_rf = sortperm(importancias_rf, rev=true) #sortperm es una función que ordena los índices de un arreglo en base a los valores del arreglo
println("Importancia de variables (Random Forest):")
for i in indices_rf
    println("Variable $(i): Importancia = $(importancias_rf[i])")
end

# Importancia de variables para XGBoost
xgb_model = XGBClassifier()
fit!(xgb_model, X_train, Y_train)
importancias_xgb = xgb_model.feature_importances_
indices_xgb = sortperm(importancias_xgb, rev=true)
println("Importancia de variables (XGBoost):")
for i in indices_xgb
    println("Variable $(i): Importancia = $(importancias_xgb[i])")
end

ErrorException: type RandomForestClassifier has no field feature_importances_

13. Realice un contraste de hipótesis para determinar cuál de los modelos y extraiga
las conclusiones. 

In [ ]:
using Pkg
Pkg.add("HypothesisTests")
using HypothesisTests


# Realizar ANOVA de una vía para comparar más de dos modelos
all_scores = vcat(rf_accuracy, xgboost_accuracy, hard_voting_accuracy, soft_voting_accuracy, lightgbm_accuracy, catboost_accuracy, knn_accuracy, svm_accuracy, mlp_accuracy, bagging_accuracy, adaboost_accuracy)
groups = repeat(["RandomForest", "XGBoost", "HardVoting", "SoftVoting", "LightGBM", "CatBoost", "KNN", "SVM", "MLP", "Bagging", "AdaBoost"], inner=[length(rf_accuracy)])

anova_result = OneWayAnova(all_scores, groups)
println(anova_result)

if anova_result.p_value < 0.05
    println("Rechazamos la hipótesis nula: Al menos un modelo es significativamente diferente.")
else
    println("No se rechaza la hipótesis nula: No hay diferencias significativas entre los modelos.")
end

# Realizar pruebas de comparación múltiple



### 6. Normalizar usando MinMaxScaler

---

## Creación de los modelos básicos

### 7. Filtrado

#### 7.1 ANOVA

#### 7.2 Mutual Information

#### 7.3 RFE con el método de LogisticRegression con una eliminación del 50% de las variables en cada pasada.

### 8. Reducción dimensionalidad

#### 8.1 PCA

#### 8.2 LDA

#### 8.3 ICA

#### 8.4 Isomap

#### 8.5 LLE

### 9. Clasificadores

#### 9.1 MLP con al menos las siguientes arquitecturas: [50], [100] [100, 50]

#### 9.2 KNN con valores de vecindario entre 1, 10 y 20

#### 9.3 SVM con el parámetro C con valores 0.1, 0.5 y 1.0

---

## Creación de los modelos ensemble

---

## Conclusiones